<div style="text-align: right" align="right"><i>Peter Norvig<br>December 2020</i></div>

# Advent of Code 2020

This year I return to [Advent of Code](https://adventofcode.com), as I did in [2016](Advent+of+Code), [17](Advent+2017), and [18](Advent-2018.ipynb). Thank you, [Eric Wastl](http://was.tl/)! This notebook describes each day's puzzle only briefly; you'll have to look at the [Advent of Code website](https://adventofcode.com/2020) if you want the full details. Each puzzle has a part 1 and a part 2.

For each day from 1 to 25, I'll write **four pieces code** with the following format (and perhaps some auxiliary code). For example, on day 3:
- `in3: List[str] = data(3)`: the day's input data, parsed into an appropriate form (here, a list of string lines).
- `def day3_1(nums): ...   `: a function that takes the day's data as input and returns the answer for part 1.
- `def day3_2(nums): ...   `: a function that takes the day's data as input and returns the answer for part 2.
- `do(3, 167, 736527114)   `: checks that `day3_1(in3) == 167` and `day3_2(in3) == 736527114`.

(During development, I can say just `do(3)` to see the output without checking it.)

# Day 0: Imports and Utility Functions

Before Day 1 I prepared (a) some imports, (b) a way to read the day's data file and to print the output, and (c) some some useful utilities:

In [1]:
import re
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, cycle, islice
from functools   import lru_cache
from typing      import Dict, Tuple, Set, List, Iterator, Optional
from sys         import maxsize

In [2]:
def data(day: int, parser=str, sep='\n') -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    sections = open(f'data/advent2020/input{day}.txt').read().rstrip().split(sep)
    return [parser(section) for section in sections]
     
def do(n, *answers) -> Tuple[Optional[int], Optional[int]]:
    "E.g., when n is 3, return [day3_1(in3), day3_2(in3)], and verify answers."
    got = [None, None]
    g = globals()
    for i in range(2):
        fname = f'day{n}_{i + 1}'
        if fname in g: 
            got[i] = g[fname](g[f'in{n}'])
            if len(answers) > i: 
                assert got[i] == answers[i], f'{fname} expected {answers[i]}; got {got[i]}'
    return got

In [3]:
def quantify(iterable, pred=bool) -> int:
    "Count the number of items in iterable for which pred is true."
    return sum(1 for item in iterable if pred(item))

def first(iterable, default=None) -> object:
    "Return first item in iterable, or default."
    return next(iter(iterable), default)

def multimap(items) -> dict:
    "Given (key, val) pairs, return {key: [val, ....], ...}."
    result = defaultdict(list)
    for (key, val) in items:
        result[key].append(val)
    return result

cat = ''.join

# Day 1: Report Repair

1. Find the two entries in your expense report (a file of integers) that sum to 2020; what do you get if you multiply them together?
2.  In your expense report, what is the product of the three entries that sum to 2020?

In [4]:
in1: Set[int] = set(data(1, int))

def day1_1(nums):
    "Find 2 distinct numbers that sum to 2020, and return their product."
    return first(x * y for x in nums 
                 for y in nums & {2020 - x} 
                 if x != y)

In [5]:
def day1_2(nums):
    "Find 3 distinct numbers that sum to 2020, and return their product."
    return first(x * y * z for x, y in combinations(nums, 2) 
                 for z in nums & {2020 - x - y} 
                 if x != z != y)

In [6]:
do(1, 787776, 262738554)

[787776, 262738554]

# Day 2: Password Philosophy

1. A password policy is of the form `1-3 b: cdefg` meaning that the password must contain 1 to 3 instances of `b`; `cdefg` is invalid under this policy. How many passwords in your input file are valid according to their policies?
-  JK! The policy actually means that exactly one of positions 1 and 3 must contain the letter `b`. How many passwords are valid according to the new interpretation of the policies?

In [7]:
def password_policy(policy) -> tuple:
    "Given policy='1-3 L: password', return (1, 3, 'L', 'password')."
    a, b, L, pw = re.findall(r'[^-:\s]+', policy)
    return (int(a), int(b), L, pw)

in2: List[tuple] = data(2, password_policy)

def valid_password(policy) -> bool: 
    "Does policy's pw have between a and b instances of letter L?"
    a, b, L, pw = policy
    return a <= pw.count(L) <= b

def day2_1(policies): return quantify(policies, valid_password)

In [8]:
def day2_2(policies): return quantify(policies, valid_password_2)

def valid_password_2(line) -> bool: 
    "Does line's pw have letter L at position a or b (1-based), but not both?"
    a, b, L, pw = line
    return (L == pw[a-1]) ^ (L == pw[b-1])

In [9]:
do(2, 383, 272)

[383, 272]

# Day 3: Toboggan Trajectory

The input file is a map of a field that looks like this:

    ..##.......
    #...#...#..
    .#....#..#.
    ..#.#...#.#
    .#...##..#.
    
where each `#` is a tree and the pattern in each row implicitly repeats to the right.

1. Starting at the top-left corner of your map and following a slope of down 1 and right 3, how many trees would you encounter?
2. What do you get if you multiply together the number of trees encountered on each of the slopes 1/1, 1/3, 1/5, 1/7, 2/1?


In [10]:
in3: List[str] = data(3)

def day3_1(rows, dx=3, dy=1, tree='#'): 
    "How many trees are on the coordinates on the slope dy/dx?"
    return quantify(row[dx * y % len(row)] == tree
                    for y, row in enumerate(rows[::dy]))

In [11]:
def day3_2(rows):
    "What is the product of the number of trees on these five slopes?"
    def t(dx, dy): return day3_1(rows, dx, dy)
    return t(1, 1) * t(3, 1) * t(5, 1) * t(7, 1) * t(1, 2) 

In [12]:
do(3, 167, 736527114)

[167, 736527114]

# Day 4: Passport Processing

The input is a file of passport data that looks like this (each passport is a series of field:value pairs separated from the next passport by a blank line):

    ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
    byr:1937 iyr:2017 cid:147 hgt:183cm

    iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884 hcl:#cfa07d byr:1929

    hcl:#ae17e1 iyr:2013
    eyr:2024 ecl:brn pid:760753108 byr:1931 hgt:179cm
    
    
1. Count the number of valid passports &mdash; those that have all seven required fields (byr, ecl, eyr, hcl, hgt, iyr, pid). 
2. Count the number of valid passports &mdash; those that have valid values for all required fields (see the rules in `valid_fields`). 

In [13]:
Passport = dict

def passport(text: str) -> Passport:
    "Make a dict of the 'key:val' entries in text."
    return Passport(re.findall(r'([a-z]+):([^\s]+)', text))

assert passport('''a:1 b:2
see:3 d:four''') == {'a': '1', 'b': '2', 'see': '3', 'd': 'four'}

in4: List[Passport] = data(4, passport, '\n\n') # Passports are separated by blank lines

required_fields = {'byr', 'ecl', 'eyr', 'hcl', 'hgt', 'iyr', 'pid'}

def valid_passport(passport) -> bool: return required_fields.issubset(passport)

def day4_1(passports): return quantify(passports, valid_passport)    

In [14]:
def day4_2(passports): return quantify(passports, valid_fields)

def valid_fields(passport) -> bool:
    '''Validate fields according to the following rules:
    byr (Birth Year) - four digits; at least 1920 and at most 2002.
    iyr (Issue Year) - four digits; at least 2010 and at most 2020.
    eyr (Expr. Year) - four digits; at least 2020 and at most 2030.
    hgt (Height) - a number followed by either cm or in:
      If cm, the number must be at least 150 and at most 193.
      If in, the number must be at least 59 and at most 76.
    hcl (Hair Color) - a '#' followed by exactly six characters 0-9 or a-f.
    ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
    pid (Passport ID) - a nine-digit number, including leading zeroes.
    cid (Country ID) - ignored, missing or not.'''
    return (valid_passport(passport)
            and all(field_validator[field](passport[field])
                    for field in required_fields))

field_validator = dict(
    byr=lambda v: 1920 <= int(v) <= 2002,
    iyr=lambda v: 2010 <= int(v) <= 2020,
    eyr=lambda v: 2020 <= int(v) <= 2030,
    hcl=lambda v: re.match('^#[0-9a-f]{6}$', v),
    ecl=lambda v: v in ('amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'),
    pid=lambda v: len(v) == 9 and v.isnumeric(),
    hgt=lambda v: ((v.endswith('cm') and 150 <= int(v[:-2]) <= 193) or
                   (v.endswith('in') and  59 <= int(v[:-2]) <=  76)))

In [15]:
do(4, 237, 172)

[237, 172]

# Day 5: Binary Boarding

The input is a list of boarding passes, such as `BFFFBBFRRR`. Each boarding pass corrsponds to a **seat ID** using an encoding where B and F stand for the back and front half of the plane; R and L stand for right and left half of a row. The encoding is the same as substituting 0 for F or L and 1 for B or R and treating the result as a binary number.

1. What is the highest seat ID on a boarding pass?
-  What is the one missing seat ID, between the minimum and maximum IDs, that is not on the list of boarding passes?

In [16]:
def seat_id(seat: str, table=str.maketrans('FLBR', '0011')) -> int:
    "Treat a seat description as a binary number; convert to int."
    return int(seat.translate(table), base=2)

assert seat_id('FBFBBFFRLR') == 357

in5: List[int] = data(5, seat_id)

day5_1 = max # Find the maximum seat id.

In [17]:
def day5_2(ids):
    "Find the one missing seat id."
    [missing] = set(range(min(ids), max(ids))) - set(ids)
    return missing

In [18]:
do(5, 906, 519)

[906, 519]

# Day 6: Custom Customs

Each passenger fills out a customs form; passengers are arranged in groups. The "yes" answer are recorded; each person on one line, each group separated by a blank line. E.g.:

    abc

    a
    b
    c

    ab
    ac
    
1. For each group, count the number of questions to which *anyone* answered "yes". What is the sum of those counts?
2. For each group, count the number of questions to which *everyone* answered "yes". What is the sum of those counts?

In [19]:
in6: List[List[str]] = data(6, str.splitlines, '\n\n')

assert in6[1] == ['arke', 'qzr', 'plmgnr', 'uriq'] # A group is a list of strs

def day6_1(groups): 
    "For each group, compute the number of letters that ANYONE got. Sum them."
    return sum(len(set(cat(group)))
               for group in groups)

In [20]:
def day6_2(groups: List[List[str]]): 
    "For each group, compute the number of letters that EVERYONE got. Sum them."
    return sum(len(set.intersection(*map(set, group)))
               for group in groups)

In [21]:
do(6, 6530, 3323)

[6530, 3323]

# Day 7: Handy Haversacks

There are strict luggage processing rules for what color bags must contain what other bags. For example:

    light red bags contain 1 bright white bag, 2 muted yellow bags.
    dark orange bags contain 3 bright white bags, 4 muted yellow bags.
    bright white bags contain 1 shiny gold bag.
    
1. How many bag colors must eventually contain at least one shiny gold bag?
2. How many individual bags must be inside your single shiny gold bag?

I wasn't quite sure, but it appears that "light red" and "dark red" are different colors.

In [22]:
Bag = str

def parse_bag_rule(line: str) -> Tuple[Bag, Dict[Bag, int]]:
    "Return (outer_bag, {inner_bag: num, ...})"
    line = re.sub(' bags?|[.]', '', line) # Remove redundant info
    outer, inner = line.split(' contain ')
    return outer, dict(map(parse_bags, inner.split(', ')))

def parse_bags(text) -> Tuple[Bag, int]:
    "Return the color and number of bags."
    n, bag = text.split(maxsplit=1)
    return bag, (0 if n == 'no' else int(n))

in7: Dict[Bag, Dict[Bag, int]] = dict(data(7, parse_bag_rule))

# in7 is of the following form:
assert (dict([parse_bag_rule("light gray bags contain 3 shiny gold bags, 1 dark red bag."),
              parse_bag_rule("shiny gold bags contain 4 bright blue bags")])
        == {'light gray': {'shiny gold': 3, 'dark red': 1},
            'shiny gold': {'bright blue': 4}})

assert parse_bags('3 muted gray') == ('muted gray', 3)

def day7_1(rules, target='shiny gold'):
    "How many colors of bags can contain the target color bag?"""
    return quantify(contains(bag, target, rules) for bag in rules)

def contains(bag, target, rules) -> bool:
    "Does this bag contain the target (perhaps recursively)?"
    contents = rules.get(bag, {})
    return (target in contents
            or any(contains(inner, target, rules) for inner in contents))

In [23]:
def day7_2(rules, target='shiny gold'): return num_contained_in(target, rules)

def num_contained_in(target, rules) -> int:
    "How many bags are contained in the target bag?"
    return sum(n + n * num_contained_in(bag, rules) 
               for (bag, n) in rules[target].items() if n > 0)

In [24]:
do(7, 103, 1469)

[103, 1469]

# Day 8: Handheld Halting

The puzzle input is a program in an assembly language with three instructions: `jmp, acc, nop`. Since there is no conditional branch instruction, a program that executes any instruction twice will infinite loop; terminating programs will execute each instruction at most once.

1. Immediately before any instruction is executed a second time, what value is in the accumulator register?
2. Fix the program so that it terminates normally by changing exactly one jmp to nop or nop to jmp. What is the value of the accumulator register after the program terminates?

In [25]:
Instruction = Tuple[str, int] # e.g. ('jmp', +4)
Program = List[Instruction]

def instruction(line: str) -> Program:
    "Parse a line of assembly code into an Instruction: an ('opcode', int) pair."
    opcode, arg = line.split()
    return opcode, int(arg)
    
in8: Program = data(8, instruction)
    
def day8_1(program):
    "Execte the program until it loops; then return accum."
    pc = accum = 0
    executed = set()
    while True:
        if pc in executed:
            return accum
        executed.add(pc)
        opcode, arg = program[pc]
        pc += 1
        if opcode == 'acc':
            accum += arg
        if opcode == 'jmp':
            pc = pc - 1 + arg

I had to think about what to do for Part 2. Do I need to make a flow graph of where the loops are? That sounds hard. But I soon realized that I can just use brute force&mdash;try every alteration of an instruction (there are only $O(n)$ of them), and run each altered program to see if it terminates (that too takes only $O(n)$ time).

In [26]:
def day8_2(program): 
    "Return the accumulator from the first altered program that terminates."
    return first(accum for (terminates, accum) in map(run, altered_programs(program))
                 if terminates)

def altered_programs(program, other=dict(jmp='nop', nop='jmp')) -> Iterator[Program]:
    "All ways to swap a nop for a jmp or vice-versa."
    for i, (opcode, arg) in enumerate(program):
        if opcode in other:
            yield [*program[:i], (other[opcode], arg), *program[i + 1:]]

def run(program) -> Tuple[bool, int]:
    "Run the program until it loops or terminates; return (terminates, accum)"
    pc = accum = 0
    executed = set()
    while 0 <= pc < len(program):
        if pc in executed:
            return False, accum # program loops
        executed.add(pc)
        opcode, arg = program[pc]
        pc += 1
        if opcode == 'acc':
            accum += arg
        if opcode == 'jmp':
            pc = pc - 1 + arg
    return True, accum # program terminates

In [27]:
do(8, 1521, 1016)

[1521, 1016]

# Day 9: Encoding Error

Given a list of numbers:

1. Find the first number in the list (after the preamble of 25 numbers) which is not the sum of two of the 25 numbers before it.
2. Find a contiguous subsequence of numbers in your list which sum to the number from step 1; add the smallest and largest numbers in this subsequence.

I could do this efficiently in $O(n)$ as in Day 1, but $n$ is so small I'll just use brute force.

In [28]:
in9 = data(9, int)

def day9_1(nums, p=25):
    """Find the first number in the list of numbers (after a preamble of p numbers) 
    which is not the sum of two of the p numbers before it."""
    return first(x for i, x in enumerate(nums) if i > p and x not in sum2(nums[i-p:i]))

def sum2(nums): return map(sum, combinations(nums, 2))

In [29]:
def day9_2(nums, target=day9_1(in9)):
    "Find a contiguous subsequence of nums that sums to target; add their max and min."
    subseq = find_subseq(nums, target)
    return max(subseq) + min(subseq)

def find_subseq(nums, target) -> deque:
    "Find a contiguous subsequence of nums that sums to target."
    subseq = deque()
    total = 0
    for x in nums:
        if total < target:
            subseq.append(x)
            total += x
        if total == target:
            return subseq
        while total > target:
            total -= subseq.popleft()

In [30]:
do(9, 776203571, 104800569)

[776203571, 104800569]

# Summary

In [31]:
%time {i: do(i) for i in range(1, 10)}

CPU times: user 226 ms, sys: 3.2 ms, total: 229 ms
Wall time: 231 ms


{1: [787776, 262738554],
 2: [383, 272],
 3: [167, 736527114],
 4: [237, 172],
 5: [906, 519],
 6: [6530, 3323],
 7: [103, 1469],
 8: [1521, 1016],
 9: [776203571, 104800569]}